Sumbitted by:  
Saurabh Kumar  
SC22B146  

# Harris Corner Detection 

### Objective
In this assignment, you will implement the Harris Corner Detection algorithm from scratch. This algorithm is fundamental in computer vision for detecting corners in images, which are useful features for image matching, object recognition, and tracking.
### Background
The Harris Corner Detection algorithm works by analyzing the local changes in image intensity to identify points that have large intensity changes in multiple directions. These points are likely to be corners. The algorithm computes a response function based on the eigenvalues of the second-moment matrix (also called the Harris matrix) for each pixel.

You will be given a skeleton code with missing parts that you need to implement. The completed code should detect corners in an input image and mark them with red dots.
You need to implement key functions:

Gaussian kernel generation

Image gradient computation

Harris matrix construction

Corner response calculation

Non-maximum suppression and thresholding

In [6]:
from __future__ import division
import numpy as np
import cv2
import matplotlib.pyplot as plt
from numpy import linalg as LA
from tqdm import tqdm

# Helper function for generating Gaussian kernel
def MyGaussianKernel(height, width, sigma):
    """
    Generate a Gaussian kernel of the specified dimensions and standard deviation.
    """
    # TODO: Implement this function
    # HINT: Use the Gaussian function formula to generate the kernel
    # The kernel should be normalized so that its sum equals 1
    ax = np.linspace(-(width // 2), width // 2, width)
    ay = np.linspace(-(height // 2), height // 2, height)
    xx, yy = np.meshgrid(ax, ay)

    kernel = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    kernel /= 2 * np.pi * sigma**2  # Gaussian normalisation factor
    kernel /= np.sum(kernel)  # Normalisation

    return kernel

In [8]:
# Helper function for downsampling an image
def DownsampleMyImage(image, factor=2):
    """
    Downsample an image by the given factor.
    """
    # TODO: Implement this function
    # HINT: You can use simple indexing or OpenCV's resize function
    
    # Using OpenCV's resize function
    height, width = image.shape[:2]
    new_size = (width // factor, height // factor)
    downsampled_image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

    return downsampled_image

In [9]:
def getGradients(image):
    """
    Compute the image gradients in x and y directions.
    Returns dx and dy - the gradients in x and y directions
    """
    # TODO: Implement this function
    # HINT: Compute the gradients by taking the difference between adjacent pixels
    # For dx, compute I(x+1,y) - I(x,y)
    # For dy, compute I(x,y+1) - I(x,y)
    
    dx = np.zeros_like(image, dtype=np.float32)
    dy = np.zeros_like(image, dtype=np.float32)

    dx[:, :-1] = image[:, 1:] - image[:, :-1]  # x-gradient
    dy[:-1, :] = image[1:, :] - image[:-1, :]  # y-gradient

    return dx, dy

In [16]:
def HarrisCornerDetection(image, kSize, sigma):
    """
    Implement the Harris Corner Detection algorithm.
    
    Args:
        image: Input image (grayscale)
        kSize: Size of the window for gradient computation
        sigma: Standard deviation for Gaussian kernel
        
    Returns:
        RMatrix: Response matrix with corner scores
    """
    # Convert to grayscale if the image is colored
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Generate Gaussian kernel
    kernel = MyGaussianKernel(kSize, kSize, sigma)
    
    # Initialize response matrix
    RMatrix = np.zeros(image.shape, dtype=np.float64)
    
    kCenterX = kSize // 2
    kCenterY = kSize // 2
    
    # Iterate over the image
    for i in tqdm(range(kCenterX, image.shape[0] - kCenterX)):
        for j in range(kCenterY, image.shape[1] - kCenterY):
            # Extract local region
            SubImage = np.zeros((kSize, kSize), dtype=np.uint8)
            
            for k in range(kSize):
                for l in range(kSize):
                    row = i - kCenterX + k
                    col = j - kCenterY + l
                    if (row >= 0 and row < image.shape[0] and col >= 0 and col < image.shape[1]):
                        SubImage[k, l] = image[row, col]
            
            # TODO: Compute gradients for the subregion
            Ix, Iy = getGradients(SubImage)
            
            # TODO: Compute products of gradients
            # HINT: Ix2 = Ix^2, Iy2 = Iy^2, Ixy = Ix*Iy
            Ix2 = Ix ** 2
            Iy2 = Iy ** 2
            Ixy = Ix * Iy
            
            # TODO: Apply Gaussian weights to the products
            # HINT: Multiply each product by the corresponding weight from the Gaussian kernel
            Ix2 = np.sum(kernel * Ix2)
            Iy2 = np.sum(kernel * Iy2)
            Ixy = np.sum(kernel * Ixy)
            
            # TODO: Construct the Harris matrix M
            # M = [sum(Ix2)  sum(Ixy)]
            #     [sum(Ixy)  sum(Iy2)]
            M = np.array([[Ix2, Ixy], 
                          [Ixy, Iy2]])
            
            # TODO: Compute the Harris response R
            # R = det(M) - k * (trace(M))^2
            # where k is typically 0.04-0.06
            # HINT: You can use eigenvalues to compute this
            k = 0.04
            detM = np.linalg.det(M)
            traceM = np.trace(M)
            R = detM - k * (traceM ** 2)

            # Store the response value
            RMatrix[i, j] = R
            
    return RMatrix

In [48]:
def NMS_and_Threshold(image, RMatrix, thresholds, window_size=3):
    """
    Perform Non-Maximum Suppression and thresholding to find corner points.
    
    Args:
        image: Original colored image
        RMatrix: Harris response matrix
        thresholds: List of threshold values for corner detection
        window_size: Size of the window for NMS
        
    Returns:
        images: List of images with marked corners for different thresholds
    """
    # TODO: Implement this function
    # HINT:
    # 1. Create copies of the original image for each threshold
    
    RMatrix = RMatrix / np.max(RMatrix) # Normalise the response matrix
    adjusted_thresholds = [thresh * np.max(RMatrix) for thresh in thresholds] # Converting to actual intensity values

    half_w = window_size // 2 # Half window size for NMS

    images = []

    # 2. For each pixel, check if it's a local maximum in its neighborhood
    # 3. If it is and its R value is above the threshold, mark it as a corner
    # 4. Return the list of images with marked corners
                        
    for threshold in adjusted_thresholds:
        # Create a copy of the original image for marking corners
        corner_image = image.copy()

        # Iterate over the image (excluding borders)
        for i in range(half_w, RMatrix.shape[0] - half_w):
            for j in range(half_w, RMatrix.shape[1] - half_w):
                local_window = RMatrix[i - half_w : i + half_w + 1, j - half_w : j + half_w + 1] # Local window around (i, j)

                if RMatrix[i, j] == np.max(local_window) and RMatrix[i, j] > threshold:
                    cv2.circle(corner_image, (j, i), radius=3, color=(0, 0, 255), thickness=-1) # Mark the corner

        # Append the modified image with marked corners
        images.append(corner_image)

    return images

In [ ]:
def main():
    # Load the image
    image_path = "board.png" # This image has clear fixed no. of corners so the output for different thresholds will be same
    image = cv2.imread(image_path, 1)
    
    # Parameters
    kSize = 7  # Window size
    sigma = 1.5  # Standard deviation for Gaussian kernel
    thresholds = [0.01, 0.2, 0.6]  # Different thresholds for corner detection
    
    # Perform Harris Corner Detection
    RMatrix = HarrisCornerDetection(image, kSize, sigma)
    
    # Save the response matrix
    np.save("RMatrix.npy", RMatrix)
    
    # Perform NMS and thresholding
    result_images = NMS_and_Threshold(image, RMatrix, thresholds)
    
    # Save the results
    for i, img in enumerate(result_images):
        cv2.imwrite(f"board_corners_T{i+1}.png", img)

    # Load the second image
    image_path2 = "icon.png"
    image2 = cv2.imread(image_path2, 1)
    
    # Perform Harris Corner Detection
    RMatrix2 = HarrisCornerDetection(image2, kSize, sigma)
    
    # Save the response matrix
    np.save("RMatrix.npy", RMatrix2)
    
    # Perform NMS and thresholding
    result_images2 = NMS_and_Threshold(image2, RMatrix2, thresholds)
    
    # Save the results
    for i, img in enumerate(result_images2):
        cv2.imwrite(f"icon_corners_T{i+1}.png", img)
    
    # Load the third image
    image_path3 = "lotus_temple.jpeg"
    image3 = cv2.imread(image_path3, 1)
    
    # Perform Harris Corner Detection
    RMatrix3 = HarrisCornerDetection(image3, kSize, sigma)
    
    # Save the response matrix
    np.save("RMatrix.npy", RMatrix3)
    
    # Perform NMS and thresholding
    result_images3 = NMS_and_Threshold(image3, RMatrix3, thresholds)
    
    # Save the results
    for i, img in enumerate(result_images3):
        cv2.imwrite(f"lotus_temple_corners_T{i+1}.png", img)
    
    print("Harris Corner Detection completed successfully!")

if __name__ == "__main__":
    main()

100%|██████████| 165/165 [00:02<00:00, 70.21it/s]


Harris Corner Detection completed successfully!
